In [1]:
import torch
import dxtb
from dxtb.typing import DD

dd: DD = {"dtype": torch.double, "device": torch.device("cpu")}

# LiH
numbers = torch.tensor([3, 1], device=dd["device"])
positions = torch.tensor([[0.0, 0.0, 0.0], [0.0, 0.0, 1.5]], **dd) # ** to use dd as kwargs 

# instantiate a calculator
calc = dxtb.calculators.GFN1Calculator(numbers, **dd) # TODO: why datatype and device? 

# Worse (for argument checking): 
calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, **dd)

In [2]:
# compute the energy
pos = positions.clone().requires_grad_(True)
energy = calc.get_energy(pos)

Total Energy: -0.67860261682577 Hartree.


# Set verbosity to 0

In [10]:
from dxtb import OutputHandler

OutputHandler.verbosity = 0

pos = positions.clone().requires_grad_(True)
energy = calc.get_energy(pos)

for verbosity in range(6):
    OutputHandler.verbosity = verbosity
    pos = positions.clone().requires_grad_(True)
    energy = calc.get_energy(pos)

OutputHandler.verbosity = 0

Total Energy: -0.67860261682577 Hartree.
SCF Energy  : -0.89655688330811 Hartree.
Total Energy: -0.67860261682577 Hartree.
Singlepoint 
 - Classicals        ... done
 - Overlap           ... done
 - Core Hamiltonian  ... done
 - Interaction Cache ... done

Starting SCF Iterations...

iter  Energy                   Delta E         Delta Pnorm     Delta q        
-----------------------------------------------------------------------------
  0   -8.77560617962661E-01    0.000000E+00    0.000000E+00    0.000000E+00
  1   -8.95077734935744E-01    1.751712E-02    2.772240E-01    2.663464E-01
  2   -8.96548332460670E-01    1.470598E-03    9.872227E-02    9.003866E-02
  3   -8.96556230914213E-01    7.898454E-06    5.901386E-03    5.292786E-03
  4   -8.96556871688813E-01    6.407746E-07    1.956400E-03    1.756005E-03
  5   -8.96556883308107E-01    1.161929E-08    2.858123E-04    2.564900E-04
  6   -8.96556883341053E-01    3.294598E-11    1.527107E-05    1.370057E-05
--------------------------

  1   -8.95077734935744E-01    1.751712E-02    2.772240E-01    2.663464E-01
  2   -8.96548332460670E-01    1.470598E-03    9.872227E-02    9.003866E-02
  3   -8.96556230914213E-01    7.898454E-06    5.901386E-03    5.292786E-03
  4   -8.96556871688813E-01    6.407746E-07    1.956400E-03    1.756005E-03
  5   -8.96556883308107E-01    1.161929E-08    2.858123E-04    2.564900E-04
  6   -8.96556883341053E-01    3.294598E-11    1.527107E-05    1.370057E-05
-----------------------------------------------------------------------------

SCF finished in 6 iterations.
SCF Energy  : -0.89655688330811 Hartree.
Total Energy: -0.67860261682577 Hartree.
Singlepoint 
 - Classicals        ... done
 - Overlap           ... done
 - Core Hamiltonian  ... done
 - Interaction Cache ... done

Starting SCF Iterations...

iter  Energy                   Delta E         Delta Pnorm     Delta q        
-----------------------------------------------------------------------------
  0   -8.77560617962661E-01    0.0

# Set the spin

In [3]:
calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, **dd)

pos = positions.clone().requires_grad_(True)
energy = calc.get_energy(pos, spin=2)


Total Energy: -0.44665978341296 Hartree.


# Quantities

In [6]:
from dxtb.config import ConfigCache
from dxtb import OutputHandler

# Compute the Density matrix, LCAO-MO coefficients and energies of the Molecular Orbitals

cache_config = ConfigCache(enabled=True, density=True, coefficients=True, mo_energies=True, fock=True)
calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, CACHE_STORE_DENSITY=True, **dd)
calc.opts.cache = cache_config
OutputHandler.verbosity = 0

density = calc.get_density(pos)
coefficients = calc.get_coefficients(pos)
energies = calc.get_mo_energies(pos)

print(f"\nshape of density: {density.shape} \nshape of coefficients: {coefficients.shape} \nshape of energies: {energies.shape}")

print(f"\nenergies: {energies}")
print(f"\ndensity: {density}")
print(f"\ncoefficients: {coefficients}")


shape of density: torch.Size([6, 6]) 
shape of coefficients: torch.Size([6, 6]) 
shape of energies: torch.Size([6])

energies: tensor([-0.4132, -0.2047, -0.1637, -0.1637, -0.0240,  0.3737],
       dtype=torch.float64, grad_fn=<AliasBackward0>)

density: tensor([[ 2.8233e-01,  4.1713e-17, -4.1713e-17,  1.3667e-01,  5.0646e-01,
          7.6675e-03],
        [ 4.1713e-17,  6.1630e-33, -6.1630e-33,  2.0193e-17,  7.4827e-17,
          1.1328e-18],
        [-4.1713e-17, -6.1630e-33,  6.1630e-33, -2.0193e-17, -7.4827e-17,
         -1.1328e-18],
        [ 1.3667e-01,  2.0193e-17, -2.0193e-17,  6.6160e-02,  2.4517e-01,
          3.7117e-03],
        [ 5.0646e-01,  7.4827e-17, -7.4827e-17,  2.4517e-01,  9.0851e-01,
          1.3754e-02],
        [ 7.6675e-03,  1.1328e-18, -1.1328e-18,  3.7117e-03,  1.3754e-02,
          2.0823e-04]], dtype=torch.float64, grad_fn=<ReshapeAliasBackward0>)

coefficients: tensor([[ 3.7572e-01, -6.5584e-01,  5.5030e-16, -1.9050e-16, -2.7768e-03,
         -1.0612e+0

In [10]:
# Integral matrices: Overlap S, Dipole, and Core Hamiltonian H0
from dxtb.integrals.wrappers import overlap, dipint, hcore

s = overlap(numbers, pos, dxtb.GFN1_XTB) # independent on the spin (uhf)
d = dipint(numbers, pos, dxtb.GFN1_XTB)
h = hcore(numbers, pos, dxtb.GFN1_XTB)

print(f"\nshape of S: {s.shape} \nshape of D: {d.shape} \nshape of H: {h.shape}")


shape of S: torch.Size([6, 6]) 
shape of D: torch.Size([3, 6, 6]) 
shape of H: torch.Size([6, 6])


In [14]:
F_cache = calc.cache["fock"]
F = torch.einsum('ij,jk,kl,lm->im', s, coefficients, torch.diag(energies), torch.inverse(coefficients))

assert torch.allclose(F, F_cache)

# Get forces via autograd or calculator directly

In [7]:
# obtain gradient (dE/dR) via autograd (Integrated Torch grad)
calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, **dd)
energy = calc.get_energy(pos)
(g,) = torch.autograd.grad(energy, pos)

Total Energy: -0.67860261682577 Hartree.


In [8]:
# Alternatively, forces can directly be requested from the calculator.
# (Don't forget to manually reset the calculator when the inputs are identical.)
calc.reset()
pos = positions.clone().requires_grad_(True)
forces = calc.get_forces(pos)

assert torch.equal(forces, -g)


Total Energy: -0.67860261682577 Hartree.


# Gradient of other matrices

In [9]:


cache_config = ConfigCache(density=True, coefficients=True, mo_energies=True)
calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, CACHE_STORE_DENSITY=True, **dd)
calc.opts.cache = cache_config

density = calc.get_density(pos)
print(f"\nshape of density: {density.shape}")
print(f"\ndensity[0]: {density[0, 0]}")

(pgrad, ) = torch.autograd.grad(density[0, 0], pos, retain_graph=True)
print(f"\nshape of pgrad: {pgrad.shape}")
print(f"\npgrad[0]: {pgrad[0]}")

Total Energy: -0.67860261682577 Hartree.



shape of density: torch.Size([6, 6])

density[0]: 0.28233208850244174

shape of pgrad: torch.Size([2, 3])

pgrad[0]: tensor([ 4.8669e-17, -1.4612e-17,  8.9727e-02], dtype=torch.float64)


# Use the jacobian

In [10]:
from torch.autograd.functional import jacobian

pos = positions.clone().requires_grad_(True)

density = calc.get_density(pos)
density_jacobian = jacobian(calc.get_density, pos)

print(f"shape of pos: {pos.shape}, shape of density: {density.shape}, shape of density_jacobian: {density_jacobian.shape}")

# Check that same as autograd
i, j = 1, 2
(pgrad, ) = torch.autograd.grad(density[i, j], pos, retain_graph=True)
print(f"jacobian [0, 0]: {density_jacobian[i, j]}")
print(f"pgrad: {pgrad}")

Total Energy: -0.67860261682577 Hartree.
Total Energy: -0.67860261682577 Hartree.


shape of pos: torch.Size([2, 3]), shape of density: torch.Size([6, 6]), shape of density_jacobian: torch.Size([6, 6, 2, 3])
jacobian [0, 0]: tensor([[ 0.0000e+00,  0.0000e+00,  3.2739e-34],
        [ 0.0000e+00,  0.0000e+00, -3.2739e-34]], dtype=torch.float64)
pgrad: tensor([[ 0.0000e+00,  0.0000e+00,  3.2739e-34],
        [ 0.0000e+00,  0.0000e+00, -3.2739e-34]], dtype=torch.float64)


# All the matrices

In [11]:
from torch.autograd.functional import jacobian

def get_s(pos):
    return overlap(numbers, pos, dxtb.GFN1_XTB)
def get_hcore(pos):
    return hcore(numbers, pos, dxtb.GFN1_XTB)
def get_p(pos):
    return calc.get_density(pos)
def get_F(pos):
    S = overlap(numbers, pos, dxtb.GFN1_XTB)
    E = calc.get_mo_energies(pos)
    C = calc.get_coefficients(pos)
    return torch.einsum('ij,jk,kl,lm->im', S, C, torch.diag(E), torch.inverse(C))

pos = positions.clone().requires_grad_(True)

S_jac = jacobian(get_s, pos)
H_jac = jacobian(get_hcore, pos)
P_jac = jacobian(get_p, pos)
F_jac = jacobian(get_F, pos)

Total Energy: -0.67860261682577 Hartree.


Total Energy: -0.67860261682577 Hartree.
Total Energy: -0.67860261682577 Hartree.
